In [3]:
import os
import json
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
# load .env
load_dotenv(find_dotenv())

True

In [2]:
# Turn off SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
directory = "data"

In [4]:
final_df = pd.read_csv(f"{directory}/raw/kbli2020.csv")
final_df.head()

,id,jenis,sumber,kode_kbli,judul,last_update,url_kbli,tags,subgolongan_kode,subgolongan_judul,subgolongan_url,golongan_kode,golongan_judul,golongan_url,golongan_pokok_kode,golongan_pokok_judul,golongan_pokok_url,kategori_kode,kategori_judul,kategori_url
0,kbli_2020_01111,kbli2020,Metadata Management System (MMS),1111,Pertanian Jagung,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"['pertanian', 'agriculture']",111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,11,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,1,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...
1,kbli_2020_01112,kbli2020,Metadata Management System (MMS),1112,Pertanian Gandum,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"['pertanian', 'agriculture']",111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,11,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,1,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...
2,kbli_2020_01113,kbli2020,Metadata Management System (MMS),1113,Pertanian Kedelai,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"['pertanian', 'agriculture']",111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,11,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,1,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...
3,kbli_2020_01114,kbli2020,Metadata Management System (MMS),1114,Pertanian Kacang Tanah,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"['pertanian', 'agriculture']",111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,11,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,1,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...
4,kbli_2020_01115,kbli2020,Metadata Management System (MMS),1115,Pertanian Kacang Hijau,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"['pertanian', 'agriculture']",111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,11,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,1,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...


In [11]:
# pemisahan KBLI menjadi beberapa entitas
def get_unique_kategori_kbli(df):
  selected_df = df.drop_duplicates(subset='kategori_kode', keep='first')
  selected_df = selected_df[['kategori_kode', 'kategori_judul', 'kategori_url']].rename(
    columns={
      'kategori_kode': 'kode',
      'kategori_judul': 'nama',
      'kategori_url': 'url'
    }
  )
  return selected_df

def get_unique_golongan_pokok_kbli(df):
  selected_df = df.drop_duplicates(subset='golongan_pokok_kode', keep='first')
  selected_df = selected_df[['golongan_pokok_kode', 'golongan_pokok_judul', 'golongan_pokok_url']].rename(
    columns={
      'golongan_pokok_kode': 'kode',
      'golongan_pokok_judul': 'nama',
      'golongan_pokok_url': 'url'
    }
  )
  return selected_df

def get_unique_golongan_kbli(df):
  selected_df = df.drop_duplicates(subset='golongan_kode', keep='first')
  selected_df = selected_df[['golongan_kode', 'golongan_judul', 'golongan_url']].rename(
    columns={
      'golongan_kode': 'kode',
      'golongan_judul': 'nama',
      'golongan_url': 'url'
    }
  )
  return selected_df

def get_unique_subgolongan_kbli(df):
  selected_df = df.drop_duplicates(subset='subgolongan_kode', keep='first')
  selected_df = selected_df[['subgolongan_kode', 'subgolongan_judul', 'subgolongan_url']].rename(
    columns={
      'subgolongan_kode': 'kode',
      'subgolongan_judul': 'nama',
      'subgolongan_url': 'url'
    }
  )
  return selected_df

def get_unique_kelompok_kbli(df):
  selected_df = df.drop_duplicates(subset='kode_kbli', keep='first')
  selected_df = selected_df[['kode_kbli', 'judul', 'url_kbli']].rename(
    columns={
      'kode_kbli': 'kode',
      'judul': 'nama',
      'url_kbli': 'url'
    }
  )
  return selected_df

def get_relasi_kelompok_kategori_kbli(df):
  selected_df = df.drop_duplicates(subset=['kode_kbli', 'kategori_kode'])
  selected_df = selected_df[['kode_kbli', 'kategori_kode']].rename(
    columns={
      'kode_kbli': 'kelompok_kbli_kode',
      'kategori_kode': 'kategori_kbli_kode'
    }
  )
  return selected_df

def get_relasi_kelompok_golongan_pokok_kbli(df):
  selected_df = df.drop_duplicates(subset=['kode_kbli', 'golongan_pokok_kode'])
  selected_df = selected_df[['kode_kbli', 'golongan_pokok_kode']].rename(
    columns={
      'kode_kbli': 'kelompok_kbli_kode',
      'golongan_pokok_kode': 'golongan_pokok_kbli_kode'
    }
  )
  return selected_df

def get_relasi_kelompok_golongan_kbli(df):
  selected_df = df.drop_duplicates(subset=['kode_kbli', 'golongan_kode'])
  selected_df = selected_df[['kode_kbli', 'golongan_kode']].rename(
    columns={
      'kode_kbli': 'kelompok_kbli_kode',
      'golongan_kode': 'golongan_kbli_kode'
    }
  )
  return selected_df

def get_relasi_kelompok_subgolongan_kbli(df):
  selected_df = df.drop_duplicates(subset=['kode_kbli', 'subgolongan_kode'])
  selected_df = selected_df[['kode_kbli', 'subgolongan_kode']].rename(
    columns={
      'kode_kbli': 'kelompok_kbli_kode',
      'subgolongan_kode': 'subgolongan_kbli_kode'
    }
  )
  return selected_df

def get_relasi_perusahaan_kelompok_kbli():
  df = pd.read_csv('../establishments/data/processed/siinas_data.csv')
  selected_df = df.drop_duplicates(subset=['id', 'kelompok_kbli'])
  selected_df['kelompok_kbli'] = selected_df['kelompok_kbli'].apply(lambda row: str(int(row)) if not pd.isna(row) else '')
  selected_df = selected_df[selected_df['kelompok_kbli'] != '']
  selected_df = selected_df[['id', 'kelompok_kbli']].rename(
    columns={
      'id': 'id_perusahaan',
      'kelompok_kbli': 'kelompok_kbli_kode',
    }
  )
  return selected_df

def save_to_csv(df, filename):
  df.to_csv(f"{directory}/processed/{filename}", index=False)
  print(f"{filename} has been saved to processed")

In [12]:
save_to_csv(get_unique_kategori_kbli(final_df), "kategori_kbli.csv")
save_to_csv(get_unique_golongan_pokok_kbli(final_df), "golongan_pokok_kbli.csv")
save_to_csv(get_unique_golongan_kbli(final_df), "golongan_kbli.csv")
save_to_csv(get_unique_subgolongan_kbli(final_df), "subgolongan_kbli.csv")
save_to_csv(get_unique_kelompok_kbli(final_df), "kelompok_kbli.csv")
save_to_csv(get_relasi_kelompok_kategori_kbli(final_df), "relasi_kelompok_kategori_kbli.csv")
save_to_csv(get_relasi_kelompok_golongan_pokok_kbli(final_df), "relasi_kelompok_golongan_pokok_kbli.csv")
save_to_csv(get_relasi_kelompok_golongan_kbli(final_df), "relasi_kelompok_golongan_kbli.csv")
save_to_csv(get_relasi_kelompok_subgolongan_kbli(final_df), "relasi_kelompok_subgolongan_kbli.csv")
save_to_csv(get_relasi_perusahaan_kelompok_kbli(), "relasi_perusahaan_kelompok_kbli.csv")

kategori_kbli.csv has been saved to processed
golongan_pokok_kbli.csv has been saved to processed
golongan_kbli.csv has been saved to processed
subgolongan_kbli.csv has been saved to processed
kelompok_kbli.csv has been saved to processed
relasi_kelompok_kategori_kbli.csv has been saved to processed
relasi_kelompok_golongan_pokok_kbli.csv has been saved to processed
relasi_kelompok_golongan_kbli.csv has been saved to processed
relasi_kelompok_subgolongan_kbli.csv has been saved to processed
relasi_perusahaan_kelompok_kbli.csv has been saved to processed


Karena tidak ada lagi data yang diproses dan sudah sesuai, maka tinggal kita copy-paste ke dalam datasets.

In [5]:
import shutil

def copy_csv(source, destination):
  # Ensure the destination directory exists
  if not os.path.exists(destination):
      os.makedirs(destination)

  # Loop through all files in the source directory
  for filename in os.listdir(source):
      # Check if the file is a CSV file
      if filename.endswith('.csv'):
          # Define full file paths
          full_filename = os.path.join(source, filename)
          if os.path.isfile(full_filename):
              # Copy the CSV file to the destination directory
              shutil.copy(full_filename, destination)

copy_csv(f"{directory}/processed", "../../datasets/kbli")